

## OpenAI Gym's Taxi solution based on https://www.gocoder.one/blog/rl-tutorial-with-openai-gym/. Test 2 different hyperparameter sets and also 1 where we do each state+action pair no more than 10 times

# 🏋️ Installing OpenAI Gym and Taxi

We'll be using the 'Taxi-v3' environment for this tutorial. To install gym (and numpy for later), run the cell below:


In [1]:
!pip install gym
!pip install numpy

Next, import gym (and additional libraries that will be useful for later):

In [2]:
import gym
import numpy as np
import random

# used to help with visualizing in Colab
from IPython.display import display, clear_output
from time import sleep

Gym contains a large library of different environments. Let's create the Taxi-v3 environment:

In [3]:
# create Taxi environment
env = gym.make('Taxi-v3')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


# 🎲 Random Agent

We'll start by implementing an agent that doesn't learn at all. Instead, it will select actions at random. This will be our baseline.

The first step is to give our agent the initial observation of the state. A state tells our agent what the current environment looks like. In Taxi, a state defines the current positions of the taxi, passenger, and pick-up and drop-off locations. Below are examples of three different states for Taxi:

![taxi states](https://www.gocoder.one/static/taxi-states-0aad1b011cf3fe07b571712f2123335c.png "Different Taxi states")

Note: Yellow = taxi, Blue letter = pickup location, Purple letter = drop-off destination

To get the initial state:

In [4]:
# create a new instance of taxi, and get the initial state
state = env.reset()

print(f"Initial state: {state}")


Initial state: 149


Next, we'll run a for-loop to cycle through the game. At each iteration, our agent will:

1. Make a random action from the action space (0 - south, 1 - north, 2 - east, 3 - west, 4 - pick-up, 5 - drop-off)
2. Receive the new state

Here's our random agent:

In [5]:
num_steps = 99
for s in range(num_steps+1):

    clear_output(wait=True)

    print(f"step: {s} out of {num_steps}")

    # sample a random action from the list of available actions
    action = env.action_space.sample()

    # perform this action on the environment
    env.step(action)

    # print the new state
    env.render()

    sleep(0.2)

# end this instance of the taxi environment
env.close()

step: 99 out of 99


When you run the cell above, you should see your agent making random moves. Not super exciting, but hopefully this helped you get familiar with the OpenAI Gym toolkit.

Next, we'll implement the Q-learning algorithm that will enable our agent to learn from rewards.

In [6]:
state_size = env.observation_space.n  # total number of states (S)
action_size = env.action_space.n      # total number of actions (A)

# initialize a qtable with 0's for all Q-values
qtable = np.zeros((state_size, action_size))

print(f"Q table: {qtable}")

Q table: [[0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 ...
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]


As our agent explores, it will update the Q-table with the Q-values it finds. To calculate our Q-values, we'll introduce the Q-learning algorithm.

Here's our implementation of the Q-learning algorithm:

In [7]:
# hyperparameters to tune
learning_rate = 0.9
discount_rate = 0.8

# dummy variables
reward = 10 # R_(t+1)
state = env.observation_space.sample()      # S_t
action = env.action_space.sample()          # A_t
new_state = env.observation_space.sample()  # S_(t+1)

# Qlearning algorithm: Q(s,a) := Q(s,a) + learning_rate * (reward + discount_rate * max Q(s',a') - Q(s,a))
qtable[state, action] += learning_rate * (reward + discount_rate * np.max(qtable[new_state,:]) - qtable[state,action])

print(f"Q-value for (state, action) pair ({state}, {action}): {qtable[state,action]}")

Q-value for (state, action) pair (165, 2): 9.0


## Exploration vs Exploitation Trade-off

We can let our agent explore to update our Q-table using the Q-learning algorithm. As our agent learns more about the environment, we can let it use this knowledge to take more optimal actions and converge faster - known as **exploitation**.

During exploitation, our agent will look at its Q-table and select the action with the highest Q-value (instead of a random action). Over time, our agent will need to explore less, and start exploiting what it knows instead.

Here's our implementation of an exploration-exploitation strategy:

In [8]:
# dummy variables
episode = random.randint(0,500)
qtable = np.random.randn(env.observation_space.sample(), env.action_space.sample())

# hyperparameters
epsilon = 1.0     # probability that our agent will explore
decay_rate = 0.01 # of epsilon

if random.uniform(0,1) < epsilon:
    # explore
    action = env.action_space.sample()
else:
    # exploit
    action = np.argmax(qtable[state,:])

# epsilon decreases exponentially --> our agent will explore less and less
epsilon = np.exp(-decay_rate*episode)

In the example above, we set some value `epsilon` between 0 and 1. If `epsilon` is 0.7, there is a 70% chance that on this step our agent will explore instead of exploit. `epsilon` exponentially decays with each step, so that our agent explores less and less over time.

# Bringing it all together

We're done with all the building blocks needed for our reinforcement learning agent. The process for training our agent will look like:

1. Initialising our Q-table with 0's for all Q-values
2. Let our agent play Taxi over a large number of games
3. Continuously update the Q-table using the Q-learning algorithm and an exploration-exploitation strategy

Here's the full implementation:

In [10]:
class bcolors:
    RED= '\u001b[31m'
    GREEN= '\u001b[32m'
    RESET= '\u001b[0m'

# create Taxi environment
env = gym.make('Taxi-v3')

# initialize q-table
state_size = env.observation_space.n
action_size = env.action_space.n
qtable = np.zeros((state_size, action_size))

# hyperparameters
#learning_rate = 0.9
#discount_rate = 0.8
#epsilon = 1.0
#decay_rate= 0.005

#1
# learning_rate = 0.6
# discount_rate = 0.9
# epsilon = 0.8
# decay_rate= 0.000

#2
learning_rate = 0.9
discount_rate = 0.8
epsilon = 1.0
decay_rate= 0.01

# 3 - Initialize visitation table
visitation_table = np.zeros((state_size, action_size))
max_visits = 10 # maximum number of visits per state-action pair

# training variables
num_episodes = 2000

# Visit each episode 99 times
max_steps = 99

print("AGENT IS TRAINING...")

for episode in range(num_episodes):

	# Reset the environment
	state = env.reset()
	step = 0
	done = False

	for step in range(max_steps):

		# Exploration-exploitation tradeoff
		if random.uniform(0,1) < epsilon:
			# Explore
      # action = env.action_space.sample()

			# 3 Get all actions that have been visited less than max_visits times for this state
			possible_actions = [action for action in range(action_size) if visitation_table[state, action] < max_visits]
			if possible_actions:
					# If there are any such actions, randomly choose from them
					action = random.choice(possible_actions)
			else:
					# Otherwise, default to a random action (or could choose another strategy)
					action = env.action_space.sample()
		else:
			# Exploit
			action = np.argmax(qtable[state,:])

		# Take an action and observe the reward
		new_state, reward, done, info = env.step(action)

		# Q-learning algorithm
		qtable[state,action] = qtable[state,action] + learning_rate * (reward + discount_rate * np.max(qtable[new_state,:])-qtable[state,action])

		# Update to our new state
		state = new_state

		# if done, finish episode
		if done == True:
			break

	# Decrease epsilon
	epsilon = np.exp(-decay_rate*episode)

# Get ready to watch our trained agent
clear_output()
print(f"Our Q-table: {qtable}")
print(f"Training completed over {num_episodes} episodes")
input("Press Enter to see our trained taxi agent")
sleep(1)
clear_output()

episodes_to_preview = 3
for episode in range(episodes_to_preview):

	# Reset the environment
	state = env.reset()
	step = 0
	done = False
	episode_rewards = 0

	for step in range(num_steps):
		# clear screen
		clear_output(wait=True)

		print(f"TRAINED AGENT")
		print(f"+++++EPISODE {episode+1}+++++")
		print(f"Step {step+1}")

		# Exploit
		action = np.argmax(qtable[state,:])

		# Take an action and observe the reward
		new_state, reward, done, info = env.step(action)

		# Accumulate our rewards
		episode_rewards += reward

		env.render()
		print("")
		if episode_rewards < 0:
			print(f"Score: {bcolors.RED}{episode_rewards}{bcolors.RESET}")
		else:
			print(f"Score: {bcolors.GREEN}{episode_rewards}{bcolors.RESET}")
		sleep(0.5)

		# Update to our new state
		state = new_state

		# if done, finish episode
		if done == True:
			break

# Close the Taxi environment
env.close()

TRAINED AGENT
+++++EPISODE 3+++++
Step 13

Score: 8
